In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '6'

In [2]:
from deeprte.experiment import Trainer
import jax
import haiku as hk

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from deeprte.config import make_config
config = make_config()
rng = hk.PRNGSequence(jax.random.PRNGKey(42))

In [4]:
e = Trainer(mode = "train", init_rng=0, config = config)

In [5]:
import tensorflow as tf
tf.nest.map_structure(lambda x: x.shape, (e.tf_data))

{'sigma': TensorShape([8, 1681, 2]),
 'psi_label': TensorShape([8, 40344]),
 'scattering_kernel': TensorShape([8, 40344, 24]),
 'self_scattering_kernel': TensorShape([8, 24, 24]),
 'boundary': TensorShape([8, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [6]:
train_input = e._build_train_input()

In [7]:
tf.nest.map_structure(lambda x: x.shape, next(train_input))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'scattering_kernel': (1, 6, 500, 24),
 'self_scattering_kernel': (1, 6, 24, 24),
 'boundary': (1, 6, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [8]:
val_input = e._build_eval_input()

In [9]:
tf.nest.map_structure(lambda x: x.shape, next(val_input))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'scattering_kernel': (1, 2, 40344, 24),
 'self_scattering_kernel': (1, 2, 24, 24),
 'boundary': (1, 2, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [10]:
ds = e._build_dummy_input()

In [11]:
tf.nest.map_structure(lambda x: x.shape, ds)

{'sigma': (1, 1, 1681, 2),
 'psi_label': (1, 1, 1),
 'scattering_kernel': (1, 1, 1, 24),
 'self_scattering_kernel': (1, 1, 24, 24),
 'boundary': (1, 1, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 1, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [12]:
model_init = jax.pmap(e.train_model.init)

In [13]:
from jaxline import utils as jl_utils
init_rng = jl_utils.bcast_local_devices(next(rng))

In [14]:
params, state = model_init(init_rng, ds)

In [15]:
apply = jax.pmap(e.train_model.apply)

In [16]:
# logits = apply(params, state, init_rng, ds)

In [17]:
e._initialize_training()

In [32]:
iterator=tf.data.Dataset.from_tensors(ds).as_numpy_iterator()

In [33]:
e._train_input = iterator

In [34]:
# scalars = e.step(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))
# scalars

In [35]:
e._eval_input = iterator

In [22]:
metrics = e.evaluate(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))

KeyError: 'psi_labels'

In [36]:
for i in e._eval_input:
    print(i["psi_labels"].shape)

KeyError: 'psi_labels'